# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import re

### 데이터 셋 읽어오기

In [2]:
# 데이터 로드
df_train = pd.read_csv("train.csv")  # 학습용 데이터
df_test = pd.read_csv("submission.csv")  # 테스트 데이터(제출 파일의 데이터)

In [3]:
print(df_train.describe())  # 수치형 데이터의 기술 통계 확인

        bant_submit  com_reg_ver_win_rate  customer_idx  \
count  59299.000000          14568.000000  59299.000000   
mean       0.634593              0.091685  27114.556333   
std        0.286066              0.150988  14653.911888   
min        0.000000              0.003788      2.000000   
25%        0.500000              0.019900  14913.000000   
50%        0.500000              0.049180  26774.000000   
75%        1.000000              0.074949  40368.500000   
max        1.000000              1.000000  47466.000000   

       historical_existing_cnt  id_strategic_ver  it_strategic_ver  \
count             13756.000000            3444.0            1121.0   
mean                 19.912184               1.0               1.0   
std                  44.697938               0.0               0.0   
min                   0.000000               1.0               1.0   
25%                   1.000000               1.0               1.0   
50%                   4.000000               1.0

In [4]:
df_train.iloc[50:80]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
50,0.75,/Aurangabad/India,AS,0.088889,47466,End-Customer,Enterprise,19.0,NaN,NaN,...,LGEIL,NaN,1,0,0.003079,0.026846,corporate / office,IT/Software,21,False
51,1.00,/AHMEDABAD/India,AS,0.088889,42158,End-Customer,SMB,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Consulting,37,False
52,1.00,/Santa Cruz do Sul - RS /Brazil,AS,0.003937,8342,End-Customer,SMB,0.0,NaN,NaN,...,LGESP,less than 3 months,1,0,0.003079,0.026846,corporate / office,NaN,38,False
53,0.75,/Dubai/U.A.E,AS,0.040816,2069,NaN,Enterprise,NaN,NaN,NaN,...,LGEGF,NaN,0,0,0.003079,0.026846,corporate / office,NaN,25,False
54,1.00,/Umm Al Quwain/U.A.E,AS,0.040816,2148,End-Customer,SMB,NaN,NaN,NaN,...,LGEGF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Manufacturing,25,False
55,1.00,/Dubai/U.A.E,AS,0.040816,38919,End-Customer,SMB,NaN,NaN,NaN,...,LGEGF,less than 3 months,1,0,0.003079,0.026846,corporate / office,NaN,25,False
56,1.00,/Centurion/South Africa,AS,0.040816,26427,Specifier/ Influencer,SMB,NaN,NaN,NaN,...,LGESA,less than 3 months,0,0,0.003079,0.026846,corporate / office,IT/Software,26,False
57,0.75,/Bogota/Colombia,AS,0.003937,13518,Specifier/ Influencer,SMB,NaN,NaN,NaN,...,LGECB,NaN,0,0,0.003079,0.026846,corporate / office,Engineering,39,False
58,1.00,/Medellín /Colombia,AS,0.003937,37960,NaN,Enterprise,42.0,NaN,NaN,...,LGECB,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,39,False
59,0.75,/Barrancabermeja/Colombia,AS,0.003937,33704,NaN,SMB,0.0,NaN,NaN,...,LGECB,NaN,0,0,0.003079,0.026846,corporate / office,NaN,40,False


In [5]:
# lead_desc_length 존재 여부
if 'lead_desc_length' in df_train.columns:
    df_train['lead_desc_length'] = np.where(df_train['lead_desc_length'] > 0, 'o', 'x')
if 'lead_desc_length' in df_test.columns:
    df_test['lead_desc_length'] = np.where(df_test['lead_desc_length'] > 0, 'o', 'x')

In [6]:
# 지역 코드에 따른 지역명 매핑
region_mapping = {
    "EU": ["LGEAG", "LGECZ", "LGEFS", "LGEDG", "LGEHS", "LGEMK", "LGEIS", "LGESC", "LGEEH", "LGEBN", "LGEWR", "LGEPL", "LGEMA", "LGEPT", "LGERO", "LGEES", "LGENO", "LGESW", "LGEUK"],
    "RC": ["LGEAK", "LGERM", "LGERI", "LGERA", "LGEUR", "LGELV"],
    "MA": ["LGEAS", "LGEEG", "LGELF", "LGESK", "LGEMC", "LGESA", "LGETU", "LGEOT", "LGEDF", "LGEGF", "LGEME", "LGEAF", "LEAO", "LGENI", "LGETK", "LGEAT", "LGESJ", "LGEEF", "LGEYK", "LGEIR"],
    "AP": ["LGEAP", "LGEQA", "LGETL", "LGECH", "LGEYT", "LGETR", "LGETA", "LGESY", "LGESH", "LGEQH", "LGEQD", "LGEPN", "LGENE", "LGEKS", "LGEHZ", "LGEHN", "LGEHK", "LGEIL", "LGEPH", "LGEVH", "LGEKR", "LGESL", "LGEIN", "LGETH", "LGEML", "LGETT", "LGEJP"],
    "NA": ["LGECI", "LGERS", "LGEMX", "LGEMS", "LGEMM", "LGEMR", "LGEUS", "LGEMU", "LGEAI"],
    "LA": ["LGEAG", "LGEBR", "LGECL", "LGEVZ", "LGECB", "LGEPS", "LGEPR", "LGESP", "LGEAR"],
    "OT": ["LGEEB", "LGELA", "LGEBT", "MA", "RC"]
}


def categorize_region(code):
    for region, codes in region_mapping.items():
        if code in codes:
            return region
    return "ETC"  

df_train['region'] = df_train['response_corporate'].apply(categorize_region)
df_test['region'] = df_test['response_corporate'].apply(categorize_region)

In [7]:
def extract_country(value):
    # 문자열이 아니면 문자열로 변환
    if not isinstance(value, str):
        value = str(value)
    # 마지막 슬래쉬 내용 -> 나라 
    match = re.search(r'\/([^\/]+)$', value)
    if match:
        return match.group(1)
    return None

df_train['customer_country'] = df_train['customer_country'].apply(extract_country)
df_test['customer_country'] = df_test['customer_country'].apply(extract_country)

In [8]:
# 제거 칼럼 
# customer_country.1의 경우 customer_country와 동일하여 제거 
# it_strategic_ver, id_strategic_ver,idit_strategic_ver 중요도 하위 삭제  
del_cols = ['customer_country.1','it_strategic_ver', 'id_strategic_ver', 'idit_strategic_ver']
df_train.drop(del_cols, axis=1, inplace=True)
df_test.drop(del_cols, axis=1, inplace=True)

In [9]:
# 언더샘플링 전 타겟 변수 'is_converted'의 분포 확인 및 언더샘플링
target_distribution = df_train['is_converted'].value_counts()
min_samples = target_distribution.min()

In [10]:
df_train_balanced = pd.concat([
    df_train[df_train['is_converted'] == True].sample(min_samples, random_state=42),
    df_train[df_train['is_converted'] == False].sample(min_samples, random_state=42)
], ignore_index=True)

In [11]:
df_train_balanced['is_converted'].value_counts()

is_converted
True     4850
False    4850
Name: count, dtype: int64

In [12]:
df_train = df_train_balanced

In [13]:
df_train.iloc[50:80]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,...,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,region
300,0.75,None,ID,NaN,47466,NaN,Enterprise,16.0,NaN,o,...,less than 3 months,0,0,NaN,NaN,NaN,NaN,159,True,NA
301,0.25,United States,IT,0.642857,11812,End Customer,SMB,0.0,NaN,o,...,NaN,0,0,0.000060,NaN,hospital & health care,Others,835,True,NA
302,0.25,United States,IT,0.642857,34283,End Customer,SMB,NaN,NaN,o,...,NaN,0,0,0.000060,NaN,hospital & health care,Others,437,True,NA
303,0.25,None,ID,0.124122,27860,NaN,SMB,3.0,support,o,...,NaN,0,0,0.000717,0.071345,hotel & accommodation,NaN,487,True,LA
304,1.00,Philippines,AS,NaN,8937,End Customer,Enterprise,NaN,operations,o,...,9 months ~ 1 year,0,0,NaN,NaN,NaN,NaN,83,True,AP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1.00,Saudi Arabia,ID,0.086957,25096,NaN,SMB,NaN,information technology,o,...,less than 3 months,0,0,0.000097,0.079412,government department,NaN,153,True,MA
376,0.25,Thailand,ID,0.118421,25096,NaN,Enterprise,NaN,purchasing,o,...,NaN,0,1,0.001183,0.049840,retail,NaN,489,True,AP
377,1.00,India,ID,NaN,33100,NaN,Enterprise,NaN,education,o,...,less than 3 months,0,0,NaN,NaN,NaN,NaN,149,True,AP
378,0.75,India,ID,NaN,25096,NaN,Enterprise,NaN,information technology,o,...,less than 3 months,0,0,NaN,NaN,NaN,NaN,155,True,AP


In [14]:
# 수치형 및 범주형 칼럼 리스트를 정의
numeric_columns = df_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_columns = df_train.select_dtypes(exclude=[np.number]).columns.tolist()

In [15]:
# 결측치 처리
numeric_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

df_train[numeric_columns] = numeric_imputer.fit_transform(df_train[numeric_columns])
df_test[numeric_columns] = numeric_imputer.transform(df_test[numeric_columns])

df_train[categorical_columns] = categorical_imputer.fit_transform(df_train[categorical_columns])
df_test[categorical_columns] = categorical_imputer.transform(df_test[categorical_columns])

In [16]:
#결측치 처리 확인 
df_train.iloc[50:80]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,...,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,region
50,0.75,Hong Kong,ID,0.172867,25096.0,End-Customer,SMB,20.661812,other,o,...,less than 3 months,0.0,0.0,0.001020,0.053848,corporate / office,Others,337.0,True,AP
51,0.25,India,AS,0.172867,25096.0,End-Customer,Enterprise,0.000000,engineering,o,...,less than 3 months,0.0,0.0,0.001020,0.053848,corporate / office,Others,19.0,True,AP
52,0.75,India,ID,0.172867,25096.0,End-Customer,SMB,20.661812,information technology,o,...,mr rajnikant is allign with the client.,0.0,0.0,0.001020,0.053848,corporate / office,Others,166.0,True,AP
53,0.50,Brazil,AS,0.003788,5847.0,End-Customer,SMB,20.661812,engineering,o,...,less than 3 months,0.0,0.0,0.000298,0.020121,residential (home),Apartment,488.0,True,LA
54,0.75,Hong Kong,ID,0.172867,25096.0,End-Customer,SMB,20.661812,information technology,o,...,less than 3 months,0.0,0.0,0.001020,0.053848,corporate / office,Others,195.0,True,AP
55,0.75,India,ID,0.172867,25096.0,End-Customer,SMB,20.661812,information technology,o,...,being followed up.,0.0,0.0,0.001020,0.053848,corporate / office,Others,155.0,True,AP
56,0.25,Taiwan,ID,0.172867,25096.0,End-Customer,SMB,20.661812,other,o,...,less than 3 months,0.0,0.0,0.001020,0.053848,corporate / office,Others,849.0,True,AP
57,0.25,None,ID,0.124122,20157.0,End-Customer,SMB,20.661812,engineering,o,...,less than 3 months,0.0,0.0,0.000717,0.071345,hotel & accommodation,Others,487.0,True,LA
58,1.00,India,ID,0.172867,25096.0,End-Customer,Enterprise,20.661812,engineering,o,...,less than 3 months,0.0,0.0,0.001020,0.053848,corporate / office,Others,314.0,True,AP
59,1.00,Brazil,AS,0.172867,25096.0,End-Customer,Enterprise,20.661812,program and project management,o,...,more than a year,0.0,0.0,0.001020,0.053848,corporate / office,Others,621.0,True,LA


## 2. 데이터 전처리

### 레이블 인코딩

In [17]:
# 범주형 데이터 레이블 인코딩
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""
    my_dict = {}
    series = series.astype(str)
    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)
    return series

In [18]:
# 레이블 인코딩할 칼럼들
# label_columns = [
#     "customer_country",
#     "business_subarea",
#     "business_area",
#     "business_unit",
#     "customer_type",
#     "enterprise",
#     "customer_job",
#     "inquiry_type",
#     "product_category",
#     "product_subcategory",
#     "product_modelname",
#     "customer_country.1",
#     "customer_position",
#     "response_corporate",
#     "expected_timeline",
# ]

# df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

# for col in label_columns:
#     df_all[col] = label_encoding(df_all[col])


다시 학습 데이터와 제출 데이터를 분리합니다.

In [19]:
for col in categorical_columns:
    df_train[col] = label_encoding(df_train[col])
    df_test[col] = label_encoding(df_test[col])

In [20]:
# 수치형 데이터 스케일링
scaler = StandardScaler()
df_train[numeric_columns] = scaler.fit_transform(df_train[numeric_columns])
df_test[numeric_columns] = scaler.transform(df_test[numeric_columns])

### 2-2. 학습, 검증 데이터 분리

In [21]:
# 특성과 레이블 분리 및 훈련 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=42,
)

In [22]:
smote = SMOTE(random_state=42)
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)
print("오버샘플링:" )
print(len(x_train_smote), len(y_train_smote))
print(y_train_smote.value_counts())

오버샘플링:
7766 7766
is_converted
1    3883
0    3883
Name: count, dtype: int64


## 3. 모델 학습

### 모델 정의 

In [23]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

### 모델 학습

In [24]:
model.fit(x_train_smote, y_train_smote)

RandomForestClassifier(random_state=42)

### 모델 성능 보기

In [25]:
# 모델 평가
def get_clf_eval(y_test, y_pred_probs):
    thresholds = np.linspace(0, 1, 100)
    best_f1 = 0
    best_threshold = 0.5
    
    for threshold in thresholds:
        y_pred = y_pred_probs >= threshold
        f1 = f1_score(y_test, y_pred)
        # 최고의 F1 점수와 임계값 찾기
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            
    # 최적의 임계값을 사용한 최종 예측
    y_pred_final = y_pred_probs >= best_threshold
    
    confusion = confusion_matrix(y_test, y_pred_final)
    accuracy = accuracy_score(y_test, y_pred_final)
    precision = precision_score(y_test, y_pred_final)
    recall = recall_score(y_test, y_pred_final)
    print("최적의 임계값:", best_threshold)
    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(best_f1))

In [26]:
y_pred_probs = model.predict_proba(x_val)[:, 1]
get_clf_eval(y_val, y_pred_probs)

최적의 임계값: 0.4646464646464647
오차행렬:
 [[874  93]
 [ 49 924]]

정확도: 0.9268
정밀도: 0.9086
재현율: 0.9496
F1: 0.9286


## 4. 제출하기

### 테스트 데이터 예측

In [27]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [28]:
test_pred = model.predict(x_test)
sum(test_pred) # True로 예측된 개수

2194

### 제출 파일 작성

In [29]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**